In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import time
from config import api_key
from scipy.stats import linregress
from pprint import pprint

In [ ]:
artist = 'ENTER NAME'

events = requests.get(f'https://app.ticketmaster.com/discovery/v2/events.json?keyword={artist}&apikey={api_key}').json()

pprint(events)